# Forensics - A. Get Started 2 - System Check and SSO to Azure

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0,  azure-cli-profile 2.1.4, Kqlmagic 0.1.95<br>
__OS Supported:__<br>
    -  Linux
__Platforms Supported:__<br>
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides code to check the common required software on the newly created DSVM and implement single sign on to Azure Resource Management for current session.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Prerequisite check

In [ ]:
# Don't need to run it again if you have already run this in provisioning notebook
#!pip install --upgrade Azure-Sentinel-Utilities

In [ ]:
import SentinelUtils
# checking Python version
check = SentinelUtils.version_management.ModuleVersionCheck()
py_check = check.validate_python('3.6.0')
if py_check.requirement_met == False:
    print('Please select Python 3.6 or Python 3.6 - AzureML at the upper right corner')
else:
    print('Please continue')

In [ ]:
# checking required packages
mods_check = check.validate_installed_modules(['Azure>=4.0.0', 'azure-cli-profile>=2.1.5', 'Kqlmagic>=0.1.95'])
for mod_info in mods_check:
    if mod_info.requirement_met == False:
        print('Please install {} {} at the following cell.'.format(mod_info.name, mod_info.required_version))

In [ ]:
# Please install required packages based on the check at last cell
#!pip install azure

In [ ]:
# this is needed only if you are going to use Device authentication
#!pip install azure-cli-profile

In [ ]:
!pip install Kqlmagic --upgrade

## SSO to Azure Resource Management

In [ ]:
# Currently Device is the only authentication that allows SSO in Sentinel Notebooks
from azure.common.client_factory import get_client_from_cli_profile
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant ''

## Other methods to log into ARM (Copy Only)

In [ ]:
# SKIP THIS IF YOU HAVE RUN SSO USING DEVICE in Part 0-B
# As alternatives to Device, you may also use [Service principal] or [User id Password]
# But these methods doesn't support SSO on Sentinel Notebooks, so you have to copy the code to individual notebooks to run 
# Copy this cell to 1. Initialize Azure Resource Management Clients, Blank cell for select an auth method
import ipywidgets as widgets
from IPython.display import display
auth_method = ['Service Principal', 'User ID/Password']
selected_auth = widgets.Dropdown(options=auth_method, value=auth_method[0],description='Auth Method:')
display(selected_auth)

In [ ]:
# SKIP THIS IF YOU HAVE RUN SSO USING DEVICE in Part 0-B
# As alternatives to Device, you may also use [Service principal] or [User id Password]
# But these methods doesn't support SSO on Sentinel Notebooks, so you have to copy the code to individual notebooks to run 
# Copy this cell to 1. Initialize Azure Resource Management Clients, Blank cell for authentication
auth = SentinelAzure.azure_aad_helper.AADHelper()
if selected_auth.value == 'Service Principal':
    client_id=input('client_id:')
    secret=input('secret:')
    tenant_id = input('tenant_id:')
    compute_client, network_client, resource_client, storage_client = auth.authenticate('Service Principal', **{'client_id': client_id, 'secret' : secret, 'tenant_id' : tenant_id, 'subscription_id' : subscription_id})
else:
    from azure.common.credentials import UserPassCredentials
    import getpass
    uid = input('User ID:')
    pwd = getpass.getpass()
    compute_client, network_client, resource_client, storage_client = auth.authenticate('User ID Password', **{'user_id': uid, 'password' : pwd, 'subscription_id' : subscription_id})

compute = SentinelAzure.azure_compute_helper.ComputeHelper(compute_client, resource_group_for_vm)
storage = SentinelAzure.azure_storage_helper.StorageHelper(storage_client)